In [1]:
# %load My_unstructured_pruning.py
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.nn.utils.prune as prune

import torchvision
import torchvision.transforms as transforms

from vgg_quant import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128

model_1 = VGG16_quant()
model_2 = VGG16_quant()
model_3 = VGG16_quant()

        

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk) # 5
    batch_size = target.size(0) # 128

    _, pred = output.topk(maxk, 1, True, True) # topk(k, dim=None, largest=True, sorted=True)
                                    # will output (max value, its index)
    pred = pred.t()               # transpose
    correct = pred.eq(target.view(1, -1).expand_as(pred))   # "-1": calculate automatically

    res = []
    for k in topk: # 1, 5
        correct_k = correct[:k].view(-1).float().sum(0)  # view(-1): make a flattened 1D tensor
        res.append(correct_k.mul_(100.0 / batch_size))   # correct: size of [maxk, batch_size]
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n    ## n is impact factor
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)



# lr = 4e-3
# weight_decay = 1e-4
# epochs = 100
# best_prec = 0
# model = model.cuda()
# criterion = nn.CrossEntropyLoss().cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# fdir = 'result/'+str(model_name)+'/model_best_valid.pth'
# epoch_parameters = 'epoch_parameters.pth'
# best_valid_parameters = 'best_valid_acc.pth'


best_valid_parameters = '4bit_VGG_best_valid_acc.pth'
checkpoint = torch.load(best_valid_parameters)
model_1.load_state_dict(checkpoint['state_dict'])
model_2.load_state_dict(checkpoint['state_dict'])

criterion = nn.CrossEntropyLoss().cuda()
model_1.eval()
model_1.cuda()
model_2.eval()
model_2.cuda()

def count_zeros_quant_only(model):
    """只统计 QuantConv2d 层的稀疏度"""
    total = 0
    zeros = 0
    for name, module in model.named_modules():
        if isinstance(module, QuantConv2d):
            weight = module.weight.data.cpu().numpy()
            total += weight.size
            zeros += (weight == 0).sum()
    return total, zeros, zeros / total if total > 0 else 0

    
print("*************Origin Version Model Test ***************")
prec_1 = validate(testloader, model_1, criterion)
total, zeros, sparsity = count_zeros_quant_only(model_1)
print(f"Total params: {total}")
print(f"Zero params: {zeros}")
print(f"Sparsity: {sparsity*100:.2f}%")

print("*************Unstructured Pruning Version Model Test ***************")
#首先进行剪枝
for layers in model_2.modules():
    if isinstance(layers, QuantConv2d):
        prune.l1_unstructured(layers, name='weight', amount=0.8)
prec_2 = validate(testloader, model_2, criterion)
print("Error Rate after Unstructured Pruning: ", prec_1 - prec_2)
total, zeros, sparsity = count_zeros_quant_only(model_2)
print(f"Total params: {total}")
print(f"Zero params: {zeros}")
print(f"Sparsity: {sparsity*100:.2f}%")

print("*************Fine-tune-Unstructured Pruning Version Model Test ***************")
PATH_prune = "0.8_ep10_vgg_pruned.pth"
checkpoint = torch.load(PATH_prune)
model_2.load_state_dict(checkpoint['model_state_dict'])
model_2.eval()
prec_3 = validate(testloader, model_2, criterion)
print("Error Rate after Unstructured Pruning: ", prec_1 - prec_3)
total, zeros, sparsity = count_zeros_quant_only(model_2)
print(f"Total params: {total}")
print(f"Zero params: {zeros}")
print(f"Sparsity: {sparsity*100:.2f}%")


=> Building model...
Files already downloaded and verified
Files already downloaded and verified
*************Origin Version Model Test ***************
Test: [0/79]	Time 0.713 (0.713)	Loss 0.3867 (0.3867)	Prec 88.281% (88.281%)
 * Prec 88.670% 
Total params: 14710464
Zero params: 0
Sparsity: 0.00%
*************Unstructured Pruning Version Model Test ***************
Test: [0/79]	Time 0.292 (0.292)	Loss 0.5321 (0.5321)	Prec 85.156% (85.156%)
 * Prec 84.940% 
Error Rate after Unstructured Pruning:  3.730000000000004
Total params: 14710464
Zero params: 11768371
Sparsity: 80.00%
*************Fine-tune-Unstructured Pruning Version Model Test ***************
Test: [0/79]	Time 0.380 (0.380)	Loss 0.3123 (0.3123)	Prec 91.406% (91.406%)
 * Prec 87.230% 
Error Rate after Unstructured Pruning:  1.4399999999999977
Total params: 14710464
Zero params: 11768371
Sparsity: 80.00%
